# Chat Models

聊天模型是语言模型的一种变体。虽然聊天模型在底层使用语言模型，但它们使用的界面略有不同。他们没有使用文本输入和文本输出API，而是使用聊天消息作为输入和输出的接口。

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug

load_dotenv(find_dotenv())
set_debug(False)



```python
pip install -qU langchain-openai
pip install -qU langchain-anthropic
pip install -qU langchain-google-vertexai
pip install -qU langchain-cohere
pip install -qU langchain-fireworks
pip install -qU langchain-mistralai
pip install -qU langchain-openai
```

聊天模型界面基于消息而不是原始文本。LangChain目前支持的消息类型有`AIMessage`、`HumanMessage`、`SystemMessage`、`FunctionMessage`和`ChatMessage`。`ChatMessage`接受任意角色参数。大多数时候，你只需要处理`HumanMessage`, `AIMessage`和`SystemMessage`

## LCEL
聊天模型实现了` Runnable interface`，这是语言链表达语言(LCEL)的基本构建块。这意味着它们支持`invoke`, `ainvoke`, `stream`, `aststream`, `batch`,` abbatch`, `aststream`日志调用。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]
chat.invoke(messages)
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

chat.batch([messages])
await chat.ainvoke(messages)
async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)
async for chunk in chat.astream_log(messages):
    print(chunk)

## generate
批处理调用，更丰富的输出您可以更进一步，使用`generate`为多组消息生成补全。这将返回带有附加`message`的`LLMResult`。除了返回的消息之外，这将包括关于每代的附加信息(例如完成原因)和关于完整API调用的附加信息(例如使用的令牌总数)。

In [ ]:
batch_messages = [
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love programming."),
    ],
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love artificial intelligence."),
    ],
]
result = chat.generate(batch_messages)
result

## 消息类型
ChatModels将消息列表作为输入并返回一条消息。有几种不同类型的消息。所有消息都有一个`role`和一个`content`属性。`role`描述了谁的消息。对于不同的角色，LangChain有不同的消息类。`content`属性描述消息的内容。这可以是一些不同的事情
- 字符串(大多数模型处理这种类型的内容)
- 字典列表(用于多模式输入，其中字典包含有关输入类型和输入位置的信息)

此外，消息还有一个`additional_kwargs`属性。在这里可以传递关于消息的附加信息。这主要用于特定于提供程序而非通用的输入参数。最著名的例子是OpenAI的`function_call `。
- HumanMessage 这表示来自用户的消息。一般只由内容组成
- AIMessage  这表示来自模型的消息。这可能会有`additional_kwargs ` -例如，如果使用OpenAI工具调用`tool_calls `。
- SystemMessage 这表示一个系统消息，它告诉模型如何行为。这通常只包括内容。并不是每个模型都支持这一点。
- FunctionMessage 这表示函数调用的结果。除了`role`和`content`之外，该消息还有一个name参数，该参数表示为产生此结果而调用的函数的名称。
- ToolMessage 这表示工具调用的结果。为了匹配OpenAI的`function`和`tool`消息类型，这与`FunctionMessage`不同。除了`role`和`content`之外，该消息还有一个`tool_call_id `参数，该参数传递了为产生此结果而调用的工具的调用id。


## Streaming
所有`chatmodel`都实现了`Runnable`接口，该接口与所有方法的默认实现一起提供。`Ainvoke`,`batch`, `abbatch`, `stream`, `astream`。这为所有`chatmodel`提供了对流的基本支持。

